In [1]:
import utils as Utils
import datetime
from bs4 import BeautifulSoup

In [2]:
# detailed olx
def olx_detailed(url):
    soup = Utils.request(url)

    details = {}

    em = soup.find("em")
    details['ID'] = Utils.rex("[0-9]+", em.find("small").text)
    details['DATES_PUBLISHED_ON'] = Utils.rex("o .*20[0-9]+", em.text)[2:]

    seen = soup.find("div", {"id":"offerbottombar"}).find_all("div")#[2]
    details['TIMES_SEEN'] = seen[len(seen) - 1].text.strip().split(':')[1]

    details['DATES_UPDATED_ON'] = None
    details['DATES_AVAILABLE_FROM'] = None

    details['OFERTA_OD'] = None
    details['RODZAJ_ZABUDOWY'] = None
    details['LICZBA_POKOI'] = None
    details['UMEBLOWANE'] = None
    details['POWIERZCHNIA'] = None
    details['CZYNSZ_DODATKOWO'] = None
    details['POZIOM'] = None

    detailed_table = soup.find("table", "details").find_all("th")
    for th in detailed_table:
        td = th.find_next_sibling("td")
        details['_'.join(th.text.strip().upper()\
                         .replace('(', '').replace(')', '').split(' '))] = td.text.strip()

    content = soup.find("div", {"id":"textContent"})
    details['DESCRIPTION'] = content.text.strip()
    details['META_DESCRIPTION'] = soup.find("meta", {"name":"description"})["content"]

    contact = soup.find("ul", {"id":"contact_methods"}).find_all("li")
    methods = []
    for method in contact:
        methods.append(method.find("i")['data-icon'])
        if method.find("i")["data-icon"] == "phone":
            methods.append(method.find("strong").text)
    details['CONTACT'] = ','.join(methods)

    return details    


In [3]:
# detailed oto
def oto_detailed(url):
    soup = Utils.request(url)
    params = soup.find("ul", "params-list")
    details = {}

    today = datetime.datetime.now().strftime("%d.%m.%Y")

    div = soup.find("div", "text-details")
    dates = div.find("div", "right").find_all("p")
    added = dates[0].text
    updated = dates[1].text
    details['DATES_UPDATED_ON'] = Utils.rex(":[ 0-9.]+", updated)[2:]
    details['DATES_PUBLISHED_ON'] = ("{}. Today is {}").format(Utils.rex(":[ a-z0-9.]+", added)[2:], \
                                                                today)
    left = div.find("div", "left")
    id = left.find("p").text.strip()
    details['ID'] = Utils.rex("Otodom:[ 0-9]+", id).split(': ')[1]
    seen = left.find_all("p")
    for p in seen:
        if Utils.rex("^Liczba wy", p.text.strip()):
            details['TIMES_SEEN'] = Utils.rex("[0-9]+", p.text.strip())
    
    details['POWIERZCHNIA'] = None
    if params.find("li", "param_m"):
        details['POWIERZCHNIA'] = params.find("li", "param_m").find("span").text

    details['LICZBA_POKOI'] = params.find("li", "param_m").find_next_sibling("li").find("strong").text
    
    details['POZIOM'] = None 
    if params.find("li", "param_floor_no"):
        details['POZIOM'] = params.find("li", "param_floor_no").find("span").text

    details['CZYNSZ_DODATKOWO'] = None
    details['RODZAJ_ZABUDOWY'] = None
    details['UMEBLOWANE'] = None
    details['DATES_AVAILABLE_FROM'] = None

    if params.find("ul", "sub-list"):
        sub = params.find("ul", "sub-list")
        lis = sub.find_all("li")
        for li in lis:
            if li.find("strong").text == "Czynsz - dodatkowo:":
                #details['CZYNSZ-EXTRA'] = if_equals_else_none(li.find("strong").text, "Czynsz - dodatkowo:", rex(":[a-z ]+", li.text)[2:])
                details['CZYNSZ_(DODATKOWO)'] = Utils.rex(":[0-9a-z�~E�~B ]+", li.text)[2:]

            if li.find("strong").text == "Rodzaj zabudowy:":
                #details['RODZAJ ZABUDOWY'] = if_equals_else_none(li.find("strong").text, "Rodzaj zabudowy:", rex(":[a-z ]+", li.text)[2:])
                details['RODZAJ_ZABUDOWY'] = Utils.rex(":[a-z ]+", li.text)[2:]

            if li.find("strong").text == "Stan wyko�~E�~Dczenia:":
                #details['UMEBLOWANE'] = if_equals_else_none(li.find("strong").text, "Stan wyko�~E�~Dczenia:", rex(":[a-z ]+", li.text)[2:])
                details['UMEBLOWANE'] = Utils.rex(":[a-z ]+", li.text)[2:]

            if li.find("strong").text == "Dost�~D�~Ypne od:":
                #details['DOSTEPNE OD'] = if_equals_else_none(li.find("strong").text, "Dost�~D�~Ypne od:", rex("[0-9]+[ a-zA-Z0-9]+", li.text))
                details['DATES_AVAILABLE_FROM'] = Utils.rex("[0-9]+[ a-zA-Z0-9]+", li.text)

    contact = soup.find("div", "box-contact-info")
    if contact.find("h6"):
        details['OFERTA_OD'] = contact.find("h6").text
    else:
        details['OFERTA_OD'] = None
    details['CONTACT'] = soup.find("span", "phone-number").text.strip()
    details['DESCRIPTION'] = soup.find("div", "text-contents").find("div", {"itemprop":"description"}).text.strip()
    details['META_DESCRIPTION'] = soup.find("meta", {"name":"description"})["content"]

    return details

In [66]:
url = "https://www.otodom.pl/oferta/lux-k-sadu-3-pokoje-gabinet-kuchnia-ID3wasE.html?utm_source=refferals&utm_medium=traffic_exchange&utm_campaign=OLX_nieruchomosci_ad#gallery[1]"
import re

In [67]:
if re.search("Stan wyko", "Stan wykonczenia:"):
    print(1)

1


In [4]:
soup = Utils.request(url)
params = soup.find("ul", "params-list")

In [5]:
oto = oto_detailed(url)
oto

{'CONTACT': '501\nWyświetl numer',
 'CZYNSZ_DODATKOWO': None,
 'DATES_AVAILABLE_FROM': None,
 'DATES_PUBLISHED_ON': '22.12.2017. Today is 30.12.2017',
 'DATES_UPDATED_ON': '28.12.2017',
 'DESCRIPTION': 'ATRAKCYJNE, CICHE I UMEBLOWANE mieszkanie w budynku z lat 50- tych, z cegły; Administracyjnie -Wola a tak na prawdę 1 przystanek do Placu Bankowego.\xa0 WSZYSTKO NOWE - w mieszkaniu nikt jeszcze nie mieszkał !!Lokal składa się z : saloniku z którego przechodzi się do gabinetu, 2 sypialni, samodzielnej kuchni oraz łazienki z wc (z kabiną prysznicową).\xa0Lokal całkowicie umeblowany i wyposażony - w każdym pomieszczeniu kanapa, w jednej sypialni podwójne łóżko; są szafy, duży telewizor, stół z krzesłami i stolik typu ława; kuchnia z pełnym wyposażeniem (włącznie ze zmywarką) - wszystko NOWE. Lokal wolny od 2 stycznia 2018 roku.\xa0Zapraszamy na prezentację ! WARTO !\xa0 \xa0 \xa0\xa0\xa0 \xa0Pośrednik odpowiedzialny zawodowo za wykonanie umowy pośrednictwa: Maria Stępień (licencja nr: 693

In [56]:
import general_olx_oto as General

TabError: inconsistent use of tabs and spaces in indentation (general_olx_oto.py, line 54)

In [68]:
import detailed_olx as Olx
import detailed_oto as Oto
import utils as Utils
import constants as Constants
import sqlite3, re, time
#import general_olx_oto as General

In [99]:
t0 = time.time()
print(type(time.localtime(t0)))
#print(Utils.from_epoch(t0))

<class 'time.struct_time'>


In [98]:
type(Utils.from_epoch(t0))

str

In [87]:
time.localtime(time.time())

time.struct_time(tm_year=2017, tm_mon=12, tm_mday=30, tm_hour=18, tm_min=46, tm_sec=59, tm_wday=5, tm_yday=364, tm_isdst=0)

In [90]:
type(time.time()) # float
time.time()

1514656146.9750373

In [100]:
t0 = time.time()
t0

1514657554.3765495

In [101]:
t1 = time.time()
t1

1514657573.620506

In [110]:
int(t1 - t0)
print(Utils.from_epoch(time.time()))
time.sleep(2 * int(t1 - t0))
print(Utils.from_epoch(time.time()))
print('Done!')

2017-12-30 19:18:48
2017-12-30 19:19:26
Done!


In [104]:
print('func started at {} and ran for {} sec.'.format(Utils.from_epoch(t1), int(t1 - t0)))

func started at 2017-12-30 19:12:53 and ran for 19 sec.


In [115]:
print('func() start: {}\nDuration: {} sec'.format(Utils.from_epoch(t0), int(t1 - t0)))

func() start: 2017-12-30 19:12:34
Duration: 19 sec


In [83]:
lines = inspect.getsourcelines(Utils.from_epoch)
#print("".join(lines[0]))
print(type(lines))
print(("".join(lines[0])))

<class 'tuple'>
def from_epoch(t0):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t0))



In [69]:
print(Olx.olx_detailed)

<function olx_detailed at 0x779ebe16cd90>


In [70]:
import inspect
lines = inspect.getsourcelines(Oto.oto_detailed)
print("".join(lines[0]))

def oto_detailed(url):
    soup = Utils.request(url)
    params = soup.find("ul", "params-list")

    details = {}
    
    today = datetime.datetime.now().strftime("%d.%m.%Y")
    
    div = soup.find("div", "text-details")
    dates = div.find("div", "right").find_all("p")
    added = dates[0].text
    updated = dates[1].text
    details['DATES_UPDATED_ON'] = Utils.rex(":[ 0-9.]+", updated)[2:]
    details['DATES_PUBLISHED_ON'] = ("{}. Today is {}").format(Utils.rex(":[ a-z0-9.]+", added)[2:], today)


    left = div.find("div", "left")
    # details['ID'] is filled in by main script
    # id = left.find("p").text.strip()
    # details['ID'] = Utils.rex("Otodom:[ 0-9]+", id).split(': ')[1]
    seen = left.find_all("p")
    
    for p in seen:
        if Utils.rex("Liczba wy.wietle. strony", p.text.strip()):
            details['TIMES_SEEN'] = Utils.rex("[0-9]+", p.text.strip())


    if params.find("li", "param_m"):     
        details['POWIERZCHNIA'] = params.find("li", "param_m").

In [27]:
conn = sqlite3.connect('/home/agigryms/Notebooks/Flat_Ads/Flat_Ads/ads.db')
conn.row_factory = sqlite3.Row
conn.isolation_level = None
c = conn.cursor()


In [28]:
db_ids = c.execute(Constants.db_ids).fetchall()
ids = []
for id in db_ids:
    ids.append(id['ID'])

In [29]:
print(len(db_ids)) # 90

90


In [55]:
ts = [1, 5, 3, 8, 4, 6]
for t in ts:
    print(t)
    if t % 2 == 0:
        print('First multiple of 2 in this list is {}.'.format(t))
        break # exits outer loop

1
5
3
8
First multiple of 2 in this list is 8.


In [212]:
url_all = "https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/?search%5Bfilter_float_m%3Afrom%5D=50"

In [74]:
results = olx_url_main(url_all)

In [75]:
len(results)

1

In [76]:
for result in results:
    print(result['ID'])

386930989


In [16]:
url_olx = "https://www.olx.pl/oferta/mieszkanie-do-wynajecia-warszawa-bemowo-CID3-IDqaNAt.html#2df33b28db"

In [24]:
olx = olx_detailed(url_olx)
olx

{'CONTACT': 'message,phone,60x xxx xxx',
 'CZYNSZ_DODATKOWO': '2 500 zł',
 'DATES_AVAILABLE_FROM': None,
 'DATES_PUBLISHED_ON': '09:11, 30 grudnia 2017',
 'DATES_UPDATED_ON': None,
 'DESCRIPTION': 'Przestronne, jasne mieszkanie 59m2, 2 pokoje, wysoki standard, wyposażone. \n\n2 pokojowe mieszkanie z kuchnią, łazienką (z ogrzewaną elektrycznie podłogą) i korytarzem w nowym budownictwie na zamkniętym, strzeżonym osiedlu (całodobowa portiernia z granitu i monitoring). Dodatkowo w mieszkaniu spory balkon.\n\nMieszkanie 59m2. Wyposażone i umeblowane;\r\n- wykończone w wysokim standardzie (w planach miało być zamieszkane przez właściciela). \n\n2 piętro (do dyspozycji mieszkańców winda), od strony wschodnio-południowej, co gwarantuje dobre nasłonecznienie a jednocześnie mieszkanie nie nagrzewa się latem. \n\nWyposażenie mieszkania obejmuje m.in.:\r\nW pełni wyposażoną kuchnię (zmywarka Bosch, nowa lodówka, płyta indukcyjna, wyciąg, mikrofalówka, piekarnik), czajnik elektryczny, talerze, garn

In [116]:
import os
# os.environ is a dictionary
print(os.environ['HOME'])

/home/agigryms


In [118]:
os.environ['MAILGUN_KEY'] = "ABC"

In [119]:
os.environ.get('MAILGUN_KEY')

'ABC'

In [19]:
import sqlite3

In [31]:
conn = sqlite3.connect("/home/agigryms/Notebooks/Flat_Ads/Flat_Ads/ads.db")
conn.row_factory = sqlite3.Row
conn.isolation_level = None
c = conn.cursor()

count = c.execute("SELECT COUNT(*) FROM ads").fetchone()


In [32]:
count[0]

162

In [213]:
def olx_url_main(url):

    # create cursor for sqlite
    conn = sqlite3.connect(Constants.db_ads)
    conn.row_factory = sqlite3.Row
    conn.isolation_level = None
    c = conn.cursor()

    c.execute(Constants.db_create_ads)

    insert_sql = Constants.db_insert_into_ads

    # get list of ad id's in db
    # ads that already exist in db will be skipped
    db_ids = c.execute(Constants.db_ids).fetchall()
    ids = []
    for id in db_ids:
        ids.append(id['ID'])

    # get keywords for filters
    patterns = Constants.patterns
    
    # start scraping
    soup = Utils.request(url)
    ads = soup.find("table", {'id':"offers_table"})

    singles = ads.find_all("tr",  "wrap")
    results = []

    # single is one ad from main url's list of ads 
    for single in singles:
        title = single.find('h3').text.strip()
        filtered_out = False

        # gen is information scraped for single from main url
        # details is information from single's detailed url
        gen = {}
        details = {}
        
        for pattern in patterns:
            if re.search(pattern, title.lower()):
                filtered_out = True
                break
                
        if not filtered_out:

            url_detailed = single.find('a')['href']

            # extract detailed info for otodom.pl ad   
            match_oto = re.search("^https?:\/\/www.otodom.pl\/", \
                                  url_detailed)
            if match_oto:
                soup_detailed = Utils.request(url_detailed)
                div = soup_detailed.find("div", "text-details")
                left = div.find("div", "left")
                soup_id = left.find("p").text.strip()
                id = Utils.rex("Otodom:[ 0-9]+", soup_id).split(': ')[1]

                # check if ad's id is not in db
                if id not in ids:
                    div = soup_detailed.find("div", "text-details")
                    dates = div.find("div", "right").find_all("p")
                    added = dates[0].text
                    
                    # check if ad is not older than 14 days; if so, it will be skipped
                    if Utils.rex(":[ a-z0-9.]+", added)[2:] != "ponad 14 dni temu":
                        gen = general_ad_info(url_detailed, title, single)
                        details = Oto.oto_detailed(gen['URL'])
                        details['ID'] = id

                        
            # extract detailed info for olx.pl ad                                          
            match_olx = re.search("^https?:\/\/www.olx.pl\/", \
                                  url_detailed)
            if match_olx:     
                soup_detailed = Utils.request(url_detailed)
                em = soup_detailed.find("em")
                id = Utils.rex("[0-9]+", em.find("small").text)
                
                # check if ad's id is not in db
                if id not in ids:
                    gen = general_ad_info(url_detailed, title, single)
                    details = Olx.olx_detailed(gen['URL'])
                    details['ID'] = id
                    
            #time.sleep(1)
            ad = {**gen, **details}
            if ad: 
                c.execute(Constants.db_insert_into_ads, (ad['TITLE'], \
                       ad['LOCATION'], \
                       ad['URL'], \
                       ad['PRICE'], \
                       ad['OFERTA_OD'], \
                       ad['CZYNSZ_DODATKOWO'], \
                       ad['POWIERZCHNIA'], \
                       ad['LICZBA_POKOI'], \
                       ad['TIMES_SEEN'], \
                       ad['DATES_PUBLISHED_ON'], \
                       ad['DATES_DB_SAVED_ON'], \
                       ad['RODZAJ_ZABUDOWY'], \
                       ad['DESCRIPTION'], \
                       ad['META_DESCRIPTION'], \
                       ad['DATES_AVAILABLE_FROM'], \
                       ad['DATES_UPDATED_ON'], \
                       ad['POZIOM'], \
                       ad['UMEBLOWANE'], \
                       ad['CONTACT'], \
                       ad['ID'], \
                      ))                
                results.append(ad)
        else:
            pass

    return results
                         

In [213]:
# draft new
def olx_url_main(url):

    # create cursor for sqlite
    conn = sqlite3.connect(Constants.db_ads)
    conn.row_factory = sqlite3.Row
    conn.isolation_level = None
    c = conn.cursor()

    c.execute(Constants.db_create_ads)

    insert_sql = Constants.db_insert_into_ads

    # get list of ad id's in db
    # ads that already exist in db will be skipped
    # "SELECT ID FROM ads GROUP BY ID"
    db_ids = c.execute(Constants.db_ids).fetchall()
    ids = []
    for id in db_ids:
        ids.append(id['ID'])

    # get keywords for filters
    patterns = Constants.patterns
    
    # start scraping
    soup = Utils.request(url)
    ads = soup.find("table", {'id':"offers_table"})

    singles = ads.find_all("tr",  "wrap")
    results = []

    # single is one ad from main url's list of ads 
    for single in singles:
        title = single.find('h3').text.strip()
        filtered_out = False

        # gen is information scraped for single from main url
        # details is information from single's detailed url
        gen = {}
        details = {}
        
        for pattern in patterns:
            if re.search(pattern, title.lower()):
                filtered_out = True
                break
                
        if not filtered_out:

            url_detailed = single.find('a')['href']

            # extract detailed info for otodom.pl ad   
            match_oto = re.search("^https?:\/\/www.otodom.pl\/", \
                                  url_detailed)
            if match_oto:
                soup_detailed = Utils.request(url_detailed)
                div = soup_detailed.find("div", "text-details")
                left = div.find("div", "left")
                soup_id = left.find("p").text.strip()
                id = Utils.rex("Otodom:[ 0-9]+", soup_id).split(': ')[1]

                # check if ad's id is not in db
                if id not in ids:
                    div = soup_detailed.find("div", "text-details")
                    dates = div.find("div", "right").find_all("p")
                    added = dates[0].text
                    
                    # check if ad is not older than 14 days; if so, it will be skipped
                    if Utils.rex(":[ a-z0-9.]+", added)[2:] != "ponad 14 dni temu":
                        gen = general_ad_info(url_detailed, title, single)
                        details = Oto.oto_detailed(gen['URL'])
                        details['ID'] = id

                        
            # extract detailed info for olx.pl ad                                          
            match_olx = re.search("^https?:\/\/www.olx.pl\/", \
                                  url_detailed)
            if match_olx:     
                soup_detailed = Utils.request(url_detailed)
                em = soup_detailed.find("em")
                id = Utils.rex("[0-9]+", em.find("small").text)
                
                # check if ad's id is not in db
                if id not in ids:
                    gen = general_ad_info(url_detailed, title, single)
                    details = Olx.olx_detailed(gen['URL'])
                    details['ID'] = id
                    
            #time.sleep(1)
            ad = {**gen, **details}
            if ad: 
                c.execute(Constants.db_insert_into_ads, (ad['TITLE'], \
                       ad['LOCATION'], \
                       ad['URL'], \
                       ad['PRICE'], \
                       ad['OFERTA_OD'], \
                       ad['CZYNSZ_DODATKOWO'], \
                       ad['POWIERZCHNIA'], \
                       ad['LICZBA_POKOI'], \
                       ad['TIMES_SEEN'], \
                       ad['DATES_PUBLISHED_ON'], \
                       ad['DATES_DB_SAVED_ON'], \
                       ad['RODZAJ_ZABUDOWY'], \
                       ad['DESCRIPTION'], \
                       ad['META_DESCRIPTION'], \
                       ad['DATES_AVAILABLE_FROM'], \
                       ad['DATES_UPDATED_ON'], \
                       ad['POZIOM'], \
                       ad['UMEBLOWANE'], \
                       ad['CONTACT'], \
                       ad['ID'], \
                      ))                
                results.append(ad)
        else:
            pass

    return results
                         

In [221]:
db_ids = "SELECT ID FROM ads"
# how to convert string with time to time and 
# look at 2 days back at the most?
db_ids = c.execute(db_ids).fetchall()
ids = []
for id in db_ids:
    ids.append(id['ID'])
ids

['51757392',
 '385686403',
 '51771856',
 '398716187',
 '51766384',
 '391451651',
 '398735611',
 '51757392',
 '385686403',
 '51771856',
 '398716187',
 '51766384',
 '391451651',
 '390524985',
 '391211165',
 '391208319',
 '390937685',
 '393560333',
 '359563090',
 '390592953',
 '374881415',
 '51770956',
 '359872846',
 '393872895',
 '395482303',
 '393917359',
 '395099613',
 '395489029',
 '395107453',
 '395100441',
 '395477783',
 '398796381',
 '384167283',
 '51775904',
 '398777599',
 '391105773',
 '391832549',
 '398770099',
 '392537979',
 '398763595',
 '398759377',
 '51618456',
 '398754859',
 '393210891',
 '376176293',
 '395752449',
 '381184309',
 '398735611',
 '51757392',
 '385686403',
 '51771856',
 '398716187',
 '51766384',
 '390524985',
 '391211165',
 '391208319',
 '390937685',
 '395089019',
 '381835675',
 '394984157',
 '399124061',
 '51793452',
 '395093863',
 '395313563',
 '395087537',
 '313042032',
 '391784541',
 '395096563',
 '395698549',
 '395109221',
 '395310607',
 '51792756',
 '5179

In [299]:
new_id = c.execute("SELECT * FROM ads WHERE ID='400218161'").fetchone()

In [300]:
print(new_id['ID'])
print(new_id['TITLE'])
print('{}: {}'.format(new_id['ID'], new_id['DATES_DB_SAVED_ON']))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
url50 = "https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/?search%5Bfilter_float_m%3Afrom%5D=50"
url60 = "https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/?search%5Bfilter_float_m%3Afrom%5D=60"

In [233]:
ids = [7, 8, 3, 4]
ad1 = {"title": "ad1", "price": "1000", 'id': 1}
ad2 = {"title": "ad2", "price": "2750", 'id': 2}
ad3 = {"title": "ad3", "price": "3000", 'id': 3}
ad4 = {"title": "ad4", "price": "4750", 'id': 4}
ads = [ad1, ad2, ad3, ad4]
ads

[{'id': 1, 'price': '1000', 'title': 'ad1'},
 {'id': 2, 'price': '2750', 'title': 'ad2'},
 {'id': 3, 'price': '3000', 'title': 'ad3'},
 {'id': 4, 'price': '4750', 'title': 'ad4'}]

In [234]:
for ad in ads:
    print(ad['id'])
    if ad['id'] in ids:
        break

1
2
3


In [275]:
import time, datetime
#t0 = time.time()

In [272]:
t9 = time.time()

In [274]:
print(from_epoch(t0))
print(from_epoch(t9))
t9 - t0

2018-01-02 20:19
2018-01-02 20:20


81.48357963562012

In [241]:
#type(new_id['DATES_DB_SAVED_ON'])
def from_epoch(t0):
    return time.strftime('%Y-%m-%d %H:%M', time.localtime(t0))

In [269]:
t0 # float

1514920777.373366

In [271]:
from_epoch(t0)

'2018-01-02 20:19'

In [276]:
datetime.datetime.fromtimestamp(t0)

datetime.datetime(2018, 1, 2, 20, 19, 37, 373366)

In [283]:
datetime.datetime.fromtimestamp(t0) > datetime.datetime.now() - datetime.timedelta(hours=48)

True

In [281]:
t0 = time.time()

In [286]:
for le in 'hackerrank':
    print(le)

h
a
c
k
e
r
r
a
n
k


In [289]:
hackerrank = 'hackerrank'
hi = 0
final = ''
for l in 'hackerworld':
    if hackerrank[hi] == l:
        final += hackerrank[hi]
        hi += 1
    else:
        continue
if final == hackerrank:
    print('YES')
else:
    print('NO')

NO


In [231]:
new = c.execute(\
"SELECT * FROM ads WHERE DATES_DB_SAVED_ON like '2018-01-02 %'").fetchall()
for n in new:
    print(n['DATES_PUBLISHED_ON'] + ': ' + n['DATES_DB_SAVED_ON'])

07:37, 2 stycznia 2018: 2018-01-02 08:00:21
07:36, 2 stycznia 2018: 2018-01-02 08:00:24
06:59, 2 stycznia 2018, ID ogłoszenia: 392096843: 2018-01-02 08:00:55
00:38, 2 stycznia 2018, ID ogłoszenia: 192058435: 2018-01-02 08:00:58
00:38, 2 stycznia 2018: 2018-01-02 08:01:02
00:32, 2 stycznia 2018: 2018-01-02 08:01:06
23:41, 1 stycznia 2018: 2018-01-02 08:01:09
01.01.2018. Today is 02.01.2018: 2018-01-02 08:01:15
22:36, 1 stycznia 2018: 2018-01-02 08:01:19
22:14, 1 stycznia 2018: 2018-01-02 08:01:22
22:02, 1 stycznia 2018: 2018-01-02 08:01:26
21:59, 1 stycznia 2018: 2018-01-02 08:01:29
01.01.2018. Today is 02.01.2018: 2018-01-02 08:01:33
21:46, 1 stycznia 2018: 2018-01-02 08:01:36
21:01, 1 stycznia 2018: 2018-01-02 08:01:42
01.01.2018. Today is 02.01.2018: 2018-01-02 08:01:47
09:04, 2 stycznia 2018: 2018-01-02 09:20:06
09:02, 2 stycznia 2018: 2018-01-02 09:20:09
08:59, 2 stycznia 2018: 2018-01-02 09:20:13
08:58, 2 stycznia 2018: 2018-01-02 09:20:16
08:58, 2 stycznia 2018: 2018-01-02 09:20:

In [ ]:
gen['DATES_DB_SAVED_ON'] = from_epoch(time.time())

In [211]:
new = c.execute(\
"SELECT * FROM ads WHERE DATES_DB_SAVED_ON like '2018-01-02 10:5%'").fetchall()
for n in new:
    print(n['TITLE'] + ': ' + n['DATES_PUBLISHED_ON'] \
          + ': ' + n['DATES_DB_SAVED_ON'])

Leszczyńska 4/ Śródmieście/ 3 pokoje/ 88 m2/ 6000: 10:43, 2 stycznia 2018: 2018-01-02 10:50:08
116m2, 4 pokoje, widok na pl.Unii Lubelskiej: 10:41, 2 stycznia 2018: 2018-01-02 10:50:11
4-pokoje, Konstruktorska 10b , TARAS: 10:41, 2 stycznia 2018: 2018-01-02 10:50:15
OKAZJA, 3 pokoje+taras i ogródek, al. Rzeczypospolitej: 10:39, 2 stycznia 2018: 2018-01-02 10:50:18
Luksusowy Apartament - długo/krótkookresowy najem: 02.01.2018. Today is 02.01.2018: 2018-01-02 10:50:23
3 pokoje, 109m2, ul. Ptasia: 10:36, 2 stycznia 2018: 2018-01-02 10:50:27
3-pokojowe mieszkanie, 70m2, ul. Bukowskiego: 10:31, 2 stycznia 2018: 2018-01-02 10:50:32


In [207]:
print('')
print('*********')
print('')


*********



In [132]:
a = {}
b = {}
ab = {**a, **b}
ab

{}

In [55]:
abs = []
if abs:
    print('not empty!')
else:
    print('empty!')

empty!


In [133]:
abs = []
abs.append(ab)

In [134]:
len(abs)

1

In [135]:
if ab:
    print('not empty')
else:
    print('empty')

empty


In [52]:
for n in new:
    print(n['TITLE'])

Przestronne, NOWE mieszkanie, osiedle Bliska Wola


In [78]:
ad1 = {"title": "ad1", "price": "2000"}
ad2 = {"title": "ad2", "price": "1750"}

In [79]:
ads = []
ads.append(ad1)
ads.append(ad2)
print(ads)

[{'title': 'ad1', 'price': '2000'}, {'title': 'ad2', 'price': '1750'}]


In [80]:
ads[1]['title']

'ad2'

In [5]:
for ad in ads:
    print(ad['price'])

2000
1750


In [103]:
total = []
for ad in ads:
    item = []
    for key, value in ad.items():
        string = '{}: {}'.format(key, value)
        item.append(string)
    total.append(item)
    total.append(['*********'])
total

[['title: ad1', 'price: 2000'],
 ['*********'],
 ['title: ad2', 'price: 1750'],
 ['*********']]

In [104]:
#r = send_email("Have a look at these new ads, darling.") # ok
#r = send_email(ads) # just keys
#r = send_email(ads[0]['title']) # ad1 - ok
#r = send_email(start) # ok, but without line breaks between items
r = send_email(total)
r.content

b'{\n  "id": "<20180101124741.1.A997F20338C1A3D5@sandbox9a3b96d505e649bbabc285f16cdb08ab.mailgun.org>",\n  "message": "Queued. Thank you."\n}'

In [14]:
def print_array_text(array):
    for t in array:
        print(t)

In [15]:
print_array_text(ads)

{'title': 'ad1', 'price': '2000'}
{'title': 'ad2', 'price': '1750'}


In [30]:
print(ads)

[{'title': 'ad1', 'price': '2000'}, {'title': 'ad2', 'price': '1750'}]


In [98]:
import requests, os

def send_email(text):
    set_mailgun_credentials()
    return requests.post(os.environ['MAILGUN_API_BASE_URL'], \
                         auth=("api", os.environ['MAILGUN_API_KEY']), \
                         data={"from": os.environ['MAILGUN_FROM'], \
                               "to": [os.environ['ALERT_RECIPIENT']], \
                               "subject": "New ads, darling.", \
                               "text": text})

In [ ]:
# filter out:
# in title:
# 

# ***
# in desc:


# None for ['CONTACT']

In [247]:
import re

In [250]:
txt = 'agencjom dziękuję -mała kilkuletnia kamienica na ul.Lubomira 3, osłonięta od ulicy Waszyngtona, mieszkanie jest na drugim piętrze, posiada 2 osobne pokoje plus salon z kuchnią, łazienka z prysznicem i fajny duży balkon otoczony koroną drzewa, nikt nie zagląda w okna ani na balkon, wyjątkowa lokalizacja, mieszkanie jest świetnie usytuowane komunikacyjnie, niedaleko tramwaje na Grochowskiej i Waszyngtona, 3 przystanki do stadionu Narodowego, 5 przystanków do Nowego Światu (12 minut), blisko zieleń parki, sklepy apteki, knajpki, kawiarenka "u Krawca", można parkować pod blokiem, zawsze jest miejsce, w pakiecie jest też miejsce na parkingu podziemnym, wyjątkowo cicho i zielono jak na centrum, zdecydowanie wyjątkowe i atrakcyjne miejsce z klimatem'

In [251]:
patterns = [".*bezpo.redni.*", ".*po.rednikom dzi.kuj.*", \
            ".*agencjom dzi.kuj.*", ".*nie wsp..pracuj. z agencj.*", \
           ".*nie wsp..pracuj. z po.rednik.*"]

In [252]:
filtered_out = False

for pattern in patterns:
            if re.search(pattern, txt.lower()):
                print(pattern)
                filtered_out = True
                break
filtered_out

.*agencjom dzi.kuj.*


True

In [256]:
li = [7, 2, 6]

for num in range(5):
    if num in li:
        break
    else:
        print(num)

0
1
